In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [ ]:
# Fit curve to eruption data
def fit_eruption(year, value, weights=None):
    if weights is None:
        weights = np.ones_like(value)

    # Translate eruption time to a small number greater than 0
    eps = 0.01
    t = year - year[0] + eps

    # Transform values to look like ~ 1/t
    data = value / value[-1]
    data = 1 / data
    data = data - 1

    # Find a least squares fit to the eruption curve using a 1/t function
    sol = least_squares(lambda x: weights * (x[0]/(t - x[1]) - data), [1, 0])

    # Transform to an "occluding" function that reduces incoming radiation
    # for time in seconds since the eruption
    def phi(t):
        # Function was fit in years.
        t = t / 365.25 / 24 / 3600
        return 1 / (sol.x[0]/(t - sol.x[1]) + 1)

    return phi

In [ ]:
# Eruption 1 fit
df = pd.read_csv('./eruption_1.csv')
year = df['date'].values
value = df['value'].values
phi_1 = fit_eruption(year, value)
# Shift time to 0 and convert to seconds
t = (year - year[0]) * 365.25 * 24 * 3600
fig, ax = plt.subplots()
ax.plot(t, value / value[-1])
ax.plot(t, phi_1(t))

In [ ]:
# Eruption 2 fit
df = pd.read_csv('./eruption_2.csv')
year = df['date'].values
value = df['value'].values
phi_2 = fit_eruption(year, value)

# Shift time to 0 and convert to seconds
t = (year - year[0]) * 365.25 * 24 * 3600
fig, ax = plt.subplots()
ax.plot(t, value / value[-1])
ax.plot(t, phi_2(t))

In [ ]:
# Take phi as the average of the two fits
phi = lambda t: 0.5 * (phi_1(t)  + phi_2(t))

In [ ]:
# Plot final function
fig, ax = plt.subplots()
years = 5
t = np.linspace(0, years * 365.25 * 24 * 3600)
ax.plot(t, phi(t))
ax.set_xlabel('Time since eruption (s)')
ax.set_ylabel('$\phi(t)$')